In [5]:
import os
import pandas as pd

# ──────────────────────────────
# 1. 기본 경로 설정
# ──────────────────────────────
base_dir = r"C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean"
output_file = os.path.join(base_dir, "csv_summary.txt")

# ──────────────────────────────
# 2. CSV 목록 불러오기
# ──────────────────────────────
csv_files = [f for f in os.listdir(base_dir) if f.lower().endswith(".csv")]

if not csv_files:
    print("❌ CSV 파일이 없습니다.")
    exit()

# ──────────────────────────────
# 3. 결과 파일 초기화
# ──────────────────────────────
with open(output_file, "w", encoding="utf-8") as f_out:
    f_out.write("📊 CSV 파일 요약 보고서\n")
    f_out.write("=" * 60 + "\n\n")

# ──────────────────────────────
# 4. 각 CSV 파일에 대한 정보 수집
# ──────────────────────────────
for file_name in csv_files:
    file_path = os.path.join(base_dir, file_name)
    print(f"처리 중: {file_name}")

    # 파일 읽기 (UTF-8 → CP949 순으로 시도)
    try:
        df = pd.read_csv(file_path, encoding="utf-8")
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding="cp949")
    except Exception as e:
        with open(output_file, "a", encoding="utf-8") as f_out:
            f_out.write(f"⚠️ {file_name} 읽기 오류: {e}\n\n")
        continue

    # ── 상위 5개 샘플만 표시 ──
    sample = df.head(5)

    # ── 결과 작성 ──
    with open(output_file, "a", encoding="utf-8") as f_out:
        f_out.write(f"📄 파일명: {file_name}\n")
        f_out.write(f"컬럼 ({len(df.columns)}개): {list(df.columns)}\n\n")
        f_out.write("샘플 5행:\n")
        f_out.write(sample.to_string(index=False))
        f_out.write("\n" + "-" * 60 + "\n\n")

# ──────────────────────────────
# 5. 완료 메시지
# ──────────────────────────────
print(f"✅ 요약 파일 생성 완료: {output_file}")


처리 중: av_contents.csv
처리 중: contents.csv
처리 중: content_ratings.csv
처리 중: game_contents.csv
처리 중: llm_recommendation_requests.csv
처리 중: naver_series_novel.csv
처리 중: platform_data.csv
처리 중: raw_items.csv
처리 중: transform_runs.csv
처리 중: users.csv
처리 중: user_preferences.csv
처리 중: user_preferred_content_types.csv
처리 중: user_preferred_genres.csv
처리 중: user_roles.csv
처리 중: webnovel_contents.csv
처리 중: webtoon_contents.csv
✅ 요약 파일 생성 완료: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\csv_summary.txt


In [9]:
import os
import io
import json
import re
import pandas as pd
import numpy as np
from datetime import datetime

# ====== 설정 ======
BASE_DIR = r"C:\Users\LG\Desktop\2025-2\A.O.D\db_export"
OUT_DIR  = os.path.join(BASE_DIR, "clean")
os.makedirs(OUT_DIR, exist_ok=True)

# 인코딩 시도 순서
ENCODINGS = ["utf-8-sig", "utf-8", "cp949", "euc-kr", "latin1"]

# 괄호 내부 콤마 치환용 (토크나이징 오류 회피)
COMMA_PLACEHOLDER = "⟐"

DATE_COL_PAT = re.compile(r"(date|_at)$", re.IGNORECASE)  # release_date, created_at 등


# ========== 유틸 ==========

def read_text_with_encodings(path: str) -> str:
    for enc in ENCODINGS:
        try:
            with io.open(path, "r", encoding=enc, errors="strict") as f:
                return f.read()
        except Exception:
            continue
    # 최후: 에러 무시 라틴1
    with io.open(path, "r", encoding="latin1", errors="ignore") as f:
        return f.read()

def protect_commas_inside_brackets(line: str) -> str:
    """{},[],() 내부의 콤마를 임시 치환해서 필드 수 불일치 방지"""
    res, stack = [], []
    for ch in line:
        if ch in "{[(":
            stack.append(ch); res.append(ch)
        elif ch in "}])":
            if stack: stack.pop()
            res.append(ch)
        else:
            if ch == "," and stack:
                res.append(COMMA_PLACEHOLDER)
            else:
                res.append(ch)
    return "".join(res)

def make_safe_temp_csv_text(raw_text: str) -> str:
    return "\n".join(protect_commas_inside_brackets(ln) for ln in raw_text.splitlines())

def try_read_csv(path: str) -> pd.DataFrame:
    """일반→치환텍스트→bad_lines skip 순서로 로드 시도"""
    raw = read_text_with_encodings(path)

    # 1) 일반 시도
    for enc in ENCODINGS + ["latin1"]:
        try:
            return pd.read_csv(io.StringIO(raw), encoding=enc, engine="python")
        except Exception:
            pass

    # 2) 괄호 내부 콤마 치환 후 재시도
    safe = make_safe_temp_csv_text(raw)
    for enc in ENCODINGS + ["latin1"]:
        try:
            df = pd.read_csv(io.StringIO(safe), encoding=enc, engine="python")
            # 치환 원복
            df = df.applymap(lambda x: x.replace(COMMA_PLACEHOLDER, ",") if isinstance(x, str) else x)
            return df
        except Exception:
            pass

    # 3) 최후: bad lines skip
    df = pd.read_csv(io.StringIO(safe), engine="python", on_bad_lines="skip")
    df = df.applymap(lambda x: x.replace(COMMA_PLACEHOLDER, ",") if isinstance(x, str) else x)
    return df

def parse_json_cell(val):
    """CSV 안 JSON 문자열(이중따옴표 이스케이프 포함)을 dict/list로 파싱(실패 시 원문 유지)"""
    if not isinstance(val, str): return val
    txt = val.strip()
    if not txt: return val
    # "{""k"":1}" -> {"k":1}
    if txt.startswith('"') and txt.endswith('"'):
        txt = txt[1:-1]
    txt = txt.replace('""', '"')
    if (txt.startswith("{") and txt.endswith("}")) or (txt.startswith("[") and txt.endswith("]")):
        try:
            return json.loads(txt)
        except Exception:
            return val
    return val

def is_jsonish_series(s: pd.Series) -> bool:
    """시리즈의 앞부분을 보고 JSON 문자열 비율이 높으면 True"""
    sample = s.dropna().astype(str).head(20).tolist()
    if not sample: return False
    cnt = 0
    for v in sample:
        v = v.strip().strip('"')
        if (v.startswith("{") and v.endswith("}")) or (v.startswith("[") and v.endswith("]")):
            cnt += 1
    return cnt >= max(3, len(sample)//4)

def normalize_df(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()

    # 1) JSON 문자열로 보이는 컬럼 파싱
    for col in out.columns:
        try:
            if out[col].dtype == "object" and is_jsonish_series(out[col]):
                out[col] = out[col].apply(parse_json_cell)
        except Exception:
            # 컬럼 단위 실패는 무시하고 진행
            pass

    # 2) 날짜 컬럼 ISO 문자열화
    for col in out.columns:
        if DATE_COL_PAT.search(col):
            try:
                out[col] = pd.to_datetime(out[col], errors="coerce").dt.strftime("%Y-%m-%d")
            except Exception:
                pass

    # 3) NaN/NaT -> None, numpy 타입 -> 파이썬 기본 타입화
    out = out.where(pd.notnull(out), None)

    return out

def df_to_json_records(df: pd.DataFrame):
    """JSON 직렬화 친화적인 records 생성"""
    # pandas 기본 dict 변환
    recs = df.to_dict(orient="records")

    # numpy 타입, Timestamp 등 처리
    def py_convert(v):
        if isinstance(v, (np.integer,)): return int(v)
        if isinstance(v, (np.floating,)) and not np.isnan(v): return float(v)
        if isinstance(v, (np.bool_,)): return bool(v)
        # Timestamp/Datetime -> ISO 문자열
        if isinstance(v, (pd.Timestamp,)): return v.strftime("%Y-%m-%dT%H:%M:%S") if pd.notna(v) else None
        return v

    def walk(o):
        if isinstance(o, dict):
            return {k: walk(py_convert(v)) for k, v in o.items()}
        if isinstance(o, list):
            return [walk(py_convert(x)) for x in o]
        return py_convert(o)

    return [walk(r) for r in recs]


# ========== 메인 ==========

def main():
    csv_files = [f for f in os.listdir(BASE_DIR) if f.lower().endswith(".csv")]
    if not csv_files:
        print("❌ CSV가 없습니다."); return

    for fn in csv_files:
        src = os.path.join(BASE_DIR, fn)
        stem = os.path.splitext(fn)[0]
        dst_json = os.path.join(OUT_DIR, f"{stem}.json")

        try:
            print(f"처리 중: {fn}")
            df = try_read_csv(src)
            df_clean = normalize_df(df)
            records = df_to_json_records(df_clean)

            with open(dst_json, "w", encoding="utf-8") as f:
                json.dump(records, f, ensure_ascii=False, indent=2)

            print(f"  ✓ 저장: {dst_json} (records: {len(records)})")
        except Exception as e:
            print(f"  ⚠️ 실패: {fn} -> {e}")

    print("\n✅ 완료: 모든 JSON은 clean/ 폴더에 저장되었습니다.")

if __name__ == "__main__":
    main()


처리 중: av_contents.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\av_contents.json (records: 13)
처리 중: contents.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\contents.json (records: 139)
처리 중: content_ratings.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\content_ratings.json (records: 0)
처리 중: game_contents.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\game_contents.json (records: 22)
처리 중: llm_recommendation_requests.csv


C:\Users\LG\AppData\Local\Temp\ipykernel_27168\4233549663.py:79: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace(COMMA_PLACEHOLDER, ",") if isinstance(x, str) else x)
C:\Users\LG\AppData\Local\Temp\ipykernel_27168\4233549663.py:79: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace(COMMA_PLACEHOLDER, ",") if isinstance(x, str) else x)
C:\Users\LG\AppData\Local\Temp\ipykernel_27168\4233549663.py:79: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace(COMMA_PLACEHOLDER, ",") if isinstance(x, str) else x)


  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\llm_recommendation_requests.json (records: 0)
처리 중: naver_series_novel.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\naver_series_novel.json (records: 0)
처리 중: platform_data.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\platform_data.json (records: 123)
처리 중: raw_items.csv


C:\Users\LG\AppData\Local\Temp\ipykernel_27168\4233549663.py:79: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace(COMMA_PLACEHOLDER, ",") if isinstance(x, str) else x)
C:\Users\LG\AppData\Local\Temp\ipykernel_27168\4233549663.py:79: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.replace(COMMA_PLACEHOLDER, ",") if isinstance(x, str) else x)


  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\raw_items.json (records: 6)
처리 중: transform_runs.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\transform_runs.json (records: 250)
처리 중: users.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\users.json (records: 0)
처리 중: user_preferences.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\user_preferences.json (records: 0)
처리 중: user_preferred_content_types.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\user_preferred_content_types.json (records: 0)
처리 중: user_preferred_genres.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\user_preferred_genres.json (records: 0)
처리 중: user_roles.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\user_roles.json (records: 0)
처리 중: webnovel_contents.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\webnovel_contents.json (records: 93)
처리 중: webtoon_contents.csv
  ✓ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\cle

In [33]:
import os
import io
import pandas as pd

BASE_DIR = r"C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean"
OUT_PATH = os.path.join(BASE_DIR, "csv_quick_summary.txt")

# 인코딩 재시도 순서
ENCODINGS = ["utf-8-sig", "utf-8", "cp949", "euc-kr", "latin1"]

def read_one_row(path):
    """여러 인코딩/옵션으로 CSV 1행만 읽기."""
    last_err = None
    for enc in ENCODINGS:
        try:
            # engine='python'으로 토크나이징에 좀 더 관대하게
            df = pd.read_csv(path, nrows=1, encoding=enc, engine="python", on_bad_lines="skip")
            return df
        except Exception as e:
            last_err = e
    # 최후: 엔진 기본으로 재시도
    try:
        df = pd.read_csv(path, nrows=1, engine="python", on_bad_lines="skip")
        return df
    except Exception as e:
        last_err = e
        raise last_err

def trim_val(v, maxlen=200):
    """샘플 값 너무 길면 자르기."""
    s = "" if (v is None or (isinstance(v, float) and pd.isna(v))) else str(v)
    s = s.replace("\r", " ").replace("\n", " ")
    if len(s) > maxlen:
        s = s[:maxlen] + "…"
    return s

def summarize_csvs():
    csv_files = [f for f in os.listdir(BASE_DIR) if f.lower().endswith(".csv")]
    csv_files.sort()

    with io.open(OUT_PATH, "w", encoding="utf-8") as out:
        out.write("📄 CSV Quick Summary\n")
        out.write("="*60 + "\n\n")

        if not csv_files:
            out.write("❌ 폴더에 CSV 파일이 없습니다.\n")
            return

        for fn in csv_files:
            path = os.path.join(BASE_DIR, fn)
            out.write(f"파일명: {fn}\n")
            try:
                df = read_one_row(path)
                cols = list(df.columns)
                out.write(f"컬럼({len(cols)}): {cols}\n")

                if df.shape[0] == 0:
                    out.write("샘플 1행: (빈 파일 또는 헤더만 존재)\n")
                else:
                    row = df.iloc[0].to_dict()
                    # 값 트리밍
                    row = {k: trim_val(v) for k, v in row.items()}
                    out.write("샘플 1행:\n")
                    # 보기 좋게 key=value 형태로 한 줄
                    row_str = " | ".join(f"{k}={row.get(k, '')}" for k in cols)
                    out.write(row_str + "\n")
            except Exception as e:
                out.write(f"⚠️ 읽기 오류: {e}\n")
            out.write("-"*60 + "\n")

    print(f"✅ 요약 파일 생성: {OUT_PATH}")

if __name__ == "__main__":
    summarize_csvs()


✅ 요약 파일 생성: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\csv_quick_summary.txt


In [26]:
import pandas as pd

# CSV 경로
csv_path = r"C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\transform_runs.csv"
df = pd.read_csv(csv_path)
df


,run_id,created_at,finished_at,status,domain,platform_name,produced_content_id,raw_id,rule_path,error
0,1,2025-10-02,2025-10-02,SUCCESS,AV,TMDB,1,1,rules/av/tmdb.yml,NaN
1,2,2025-10-02,2025-10-02,SUCCESS,AV,TMDB,2,2,rules/av/tmdb.yml,NaN
2,3,2025-10-02,2025-10-02,SUCCESS,AV,TMDB,3,3,rules/av/tmdb.yml,NaN
3,4,2025-10-02,2025-10-02,SUCCESS,AV,TMDB,4,4,rules/av/tmdb.yml,NaN
4,5,2025-10-02,2025-10-02,SUCCESS,AV,TMDB,5,5,rules/av/tmdb.yml,NaN
...,...,...,...,...,...,...,...,...,...,...
245,246,2025-10-02,2025-10-02,SUCCESS,WEBNOVEL,KakaoPage,240,246,rules/webnovel/kakaopage.yml,NaN
246,247,2025-10-02,2025-10-02,SUCCESS,WEBNOVEL,KakaoPage,241,247,rules/webnovel/kakaopage.yml,NaN
247,248,2025-10-02,2025-10-02,SUCCESS,WEBNOVEL,KakaoPage,242,248,rules/webnovel/kakaopage.yml,NaN
248,249,2025-10-02,2025-10-02,SUCCESS,WEBNOVEL,KakaoPage,243,249,rules/webnovel/kakaopage.yml,NaN


# 이분 그래프 생성

In [37]:
# -*- coding: utf-8 -*-

import os
import re
import pandas as pd

BASE = r"C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean"
PATH_CONTENTS = os.path.join(BASE, "contents.csv")
PATH_AV       = os.path.join(BASE, "av_contents.csv")
PATH_GAME     = os.path.join(BASE, "game_contents.csv")
PATH_WEBNOVEL = os.path.join(BASE, "webnovel_contents.csv")  # 있으면 사용

OUT_NODES     = os.path.join(BASE, "graph_nodes.csv")              # 콘텐츠 노드
OUT_RAWGEN    = os.path.join(BASE, "content_raw_genres.csv")       # 원본 장르 세로표
OUT_META      = os.path.join(BASE, "meta_nodes.csv")               # 메타노드 목록
OUT_EDGES_BI  = os.path.join(BASE, "graph_edges_bipartite.csv")    # 콘텐츠→메타 엣지

# 엣지 가중치(필요 시 조정)
DOMAIN_EDGE_WEIGHT = 1.0
GENRE_EDGE_WEIGHT  = 1.0

ENCODINGS = ["utf-8-sig","utf-8","cp949","euc-kr","latin1"]
def read_csv_retry(path, **kwargs):
    last = None
    for enc in ENCODINGS:
        try:
            return pd.read_csv(path, encoding=enc, **kwargs)
        except Exception as e:
            last = e
    raise last

def safe_read(path):
    if not os.path.exists(path):
        print(f"⚠️ 없음: {path}")
        return None
    try:
        return read_csv_retry(path)
    except Exception as e:
        print(f"⚠️ 읽기 실패: {path} -> {e}")
        return None

def add_prefix_except_key(df, prefix, key="content_id"):
    if df is None: return None
    return df.rename(columns={c: (c if c==key else f"{prefix}{c}") for c in df.columns})

def norm_for_id(s: str) -> str:
    """
    메타노드 ID용 간단 정규화 (공백/구두점 제거, 소문자)
    예: 'Science Fiction' -> 'science_fiction'
    """
    if s is None: return ""
    s = str(s).strip().lower()
    # 한/영/숫자 외는 공백 치환
    s = re.sub(r"[^\w\s가-힣]", " ", s)
    s = "_".join(s.split())
    return s

# 1) 콘텐츠 노드 생성(그대로 유지)
def build_nodes():
    contents = safe_read(PATH_CONTENTS)
    if contents is None or contents.empty:
        raise RuntimeError("❌ contents.csv 비어있음/로드 실패")
    if "content_id" not in contents.columns or "domain" not in contents.columns:
        raise RuntimeError("❌ contents.csv에 content_id/domain 컬럼 필요")

    av   = add_prefix_except_key(safe_read(PATH_AV), "av_")
    game = add_prefix_except_key(safe_read(PATH_GAME), "game_")
    web  = add_prefix_except_key(safe_read(PATH_WEBNOVEL), "webnovel_")

    def typed_left_merge(left, right, key="content_id"):
        if right is None or right.empty:
            return left.copy()
        L = left.copy(); R = right.copy()
        if key in L.columns: L[key] = pd.to_numeric(L[key], errors="coerce").astype("Int64")
        if key in R.columns: R[key] = pd.to_numeric(R[key], errors="coerce").astype("Int64")
        return L.merge(R, on=key, how="left")

    parts = []
    doms = contents["domain"].dropna().unique().tolist()
    if "AV" in doms:
        sub = contents[contents["domain"]=="AV"].copy()
        sub = typed_left_merge(sub, av); parts.append(sub)
    if "GAME" in doms:
        sub = contents[contents["domain"]=="GAME"].copy()
        sub = typed_left_merge(sub, game); parts.append(sub)
    if "WEBNOVEL" in doms:
        sub = contents[contents["domain"]=="WEBNOVEL"].copy()
        sub = typed_left_merge(sub, web); parts.append(sub)

    known = {"AV","GAME","WEBNOVEL"}
    nodes = pd.concat(parts, ignore_index=True) if parts else contents[contents["domain"].isin(known)].copy()
    nodes = nodes.drop_duplicates(subset=["content_id"], keep="first")

    # 보기 좋게 정렬
    first = [c for c in ["content_id","domain","master_title","original_title","release_year",
                         "poster_image_url","created_at","updated_at","synopsis"] if c in nodes.columns]
    nodes = nodes[first + [c for c in nodes.columns if c not in first]]

    nodes.to_csv(OUT_NODES, index=False, encoding="utf-8-sig")
    print(f"✅ graph_nodes 저장: {OUT_NODES} (rows={len(nodes)})")
    return nodes

# 2) 원본 장르 세로표(content_id, source, raw_genre)
def build_raw_genres(nodes):
    valid_ids = set(pd.to_numeric(nodes["content_id"], errors="coerce").dropna().astype(int))
    rows = []

    # AV: TMDB 장르 (genres.tmdb_genres.N.name)
    av = safe_read(PATH_AV)
    if av is not None and not av.empty:
        name_cols = [c for c in av.columns if c.startswith("genres.tmdb_genres.") and c.endswith(".name")]
        if name_cols:
            tmp = av.melt(id_vars=["content_id"], value_vars=name_cols, value_name="raw_genre").dropna(subset=["raw_genre"])
            tmp["raw_genre"] = tmp["raw_genre"].astype(str).str.strip()
            tmp = tmp[(tmp["raw_genre"]!="") & (tmp["content_id"].isin(valid_ids))]
            tmp["source"] = "tmdb"
            rows.append(tmp[["content_id","source","raw_genre"]])

    # GAME: Steam 장르 (genres_str: "A;B;C")
    game = safe_read(PATH_GAME)
    if game is not None and not game.empty and "genres_str" in game.columns:
        g2 = game.dropna(subset=["genres_str"]).copy()
        g2["raw_genre"] = g2["genres_str"].astype(str).str.split(r"\s*;\s*")
        g2 = g2.explode("raw_genre").dropna(subset=["raw_genre"])
        g2["raw_genre"] = g2["raw_genre"].astype(str).str.strip()
        g2 = g2[(g2["raw_genre"]!="") & (g2["content_id"].isin(valid_ids))]
        g2["source"] = "steam"
        rows.append(g2[["content_id","source","raw_genre"]])

    # WEBNOVEL: 장르 컬럼이 있다면 분해 (예: 'genres'가 "판타지;로맨스")
    web = safe_read(PATH_WEBNOVEL)
    if web is not None and not web.empty:
        # 후보 컬럼들 중 존재하는 것 선택
        cand_cols = [c for c in ["genres","genre","genre_str"] if c in web.columns]
        if cand_cols:
            col = cand_cols[0]
            w2 = web.dropna(subset=[col]).copy()
            # 구분자: ; , / | (혼용 방지용 정규식)
            w2["raw_genre"] = w2[col].astype(str).str.split(r"\s*[;|/,]\s*")
            w2 = w2.explode("raw_genre").dropna(subset=["raw_genre"])
            w2["raw_genre"] = w2["raw_genre"].astype(str).str.strip()
            if "content_id" in w2.columns:
                w2 = w2[w2["raw_genre"]!=""]
                w2 = w2[w2["content_id"].isin(valid_ids)]
                w2["source"] = "webnovel"
                rows.append(w2[["content_id","source","raw_genre"]])

    df = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame(columns=["content_id","source","raw_genre"])
    if not df.empty:
        df["content_id"] = pd.to_numeric(df["content_id"], errors="coerce").astype("Int64")
        df = df[df["content_id"].notna()]
        df["content_id"] = df["content_id"].astype(int)

    df = df.drop_duplicates().reset_index(drop=True)
    df.to_csv(OUT_RAWGEN, index=False, encoding="utf-8-sig")
    print(f"✅ content_raw_genres 저장: {OUT_RAWGEN} (rows={len(df)})")
    return df

# 3) 메타노드 + 이분 엣지 생성 (도메인/장르 모두 '원본' 기준)
def build_bipartite(nodes, raw_genres):
    # 메타노드: 도메인
    domain_nodes = sorted(nodes["domain"].dropna().unique().tolist())

    meta_rows = []
    for d in domain_nodes:
        meta_rows.append({"meta_id": f"DOM:{d}", "meta_type": "domain", "label": d, "source": ""})

    # 메타노드: 장르 (네임스페이스 적용)
    if raw_genres is not None and not raw_genres.empty:
        # (source, raw_genre_norm)별 하나의 메타노드
        rg = raw_genres.copy()
        rg["raw_genre_norm"] = rg["raw_genre"].astype(str).str.strip()
        rg["raw_genre_norm"] = rg["raw_genre_norm"].replace({"": None})
        rg = rg.dropna(subset=["raw_genre_norm"])
        uniq = rg[["source","raw_genre_norm"]].drop_duplicates()
        for src, g in uniq.itertuples(index=False):
            meta_id = f"GEN:{src}:{norm_for_id(g)}" if g else None
            if meta_id:
                meta_rows.append({"meta_id": meta_id, "meta_type": "genre", "label": g, "source": src})

    df_meta = pd.DataFrame(meta_rows, columns=["meta_id","meta_type","label","source"]).drop_duplicates()
    df_meta.to_csv(OUT_META, index=False, encoding="utf-8-sig")
    print(f"✅ meta_nodes 저장: {OUT_META} (rows={len(df_meta)})")

    # 이분 엣지: 콘텐츠 → 도메인
    edges = []
    sub = nodes[["content_id","domain"]].dropna()
    for cid, dom in sub.itertuples(index=False):
        edges.append({
            "src_content_id": int(cid),
            "dst_meta_id":    f"DOM:{dom}",
            "edge_type":      "content-domain",
            "weight":         DOMAIN_EDGE_WEIGHT
        })

    # 이분 엣지: 콘텐츠 → (source별 원본장르)
    if raw_genres is not None and not raw_genres.empty:
        for cid, src, g in raw_genres[["content_id","source","raw_genre"]].itertuples(index=False):
            g_norm = norm_for_id(g)
            if not g_norm:
                continue
            meta_id = f"GEN:{src}:{g_norm}"
            edges.append({
                "src_content_id": int(cid),
                "dst_meta_id":    meta_id,
                "edge_type":      f"content-genre-{src}",
                "weight":         GENRE_EDGE_WEIGHT
            })

    df_edges = pd.DataFrame(edges, columns=["src_content_id","dst_meta_id","edge_type","weight"]).drop_duplicates()
    df_edges.to_csv(OUT_EDGES_BI, index=False, encoding="utf-8-sig")
    print(f"✅ graph_edges_bipartite 저장: {OUT_EDGES_BI} (edges={len(df_edges)})")
    return df_meta, df_edges

def main():
    nodes = build_nodes()
    rawg  = build_raw_genres(nodes)
    build_bipartite(nodes, rawg)

if __name__ == "__main__":
    main()


✅ graph_nodes 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\graph_nodes.csv (rows=139)
✅ content_raw_genres 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\content_raw_genres.csv (rows=33)
✅ meta_nodes 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\meta_nodes.csv (rows=12)
✅ graph_edges_bipartite 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\graph_edges_bipartite.csv (edges=172)


# 아이템-아이템 그래프 생성

In [39]:
# -*- coding: utf-8 -*-


import os
import math
import pandas as pd
from collections import defaultdict

BASE = r"C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean"
PATH_NODES   = os.path.join(BASE, "graph_nodes.csv")
PATH_RAWGEN  = os.path.join(BASE, "content_raw_genres.csv")
OUT_EDGES    = os.path.join(BASE, "graph_edges_item_item.csv")

# ========= 하이퍼파라미터(조절 포인트) =========
DOMAIN_BASE_WEIGHT    = 1.0   # 같은 도메인 공유 기여치
GENRE_BASE_WEIGHT     = 1.0   # 같은 '도메인별 원본장르' 공유 기여치
ALPHA                 = 0.7   # 허브 완화 지수(0=완화 없음, 1=강한 완화). 보통 0.5~0.8
USE_IDF               = True  # 메타노드 정보량 가중치(log(1+N/deg)) 사용
TOPK_PER_ITEM         = 100   # 아이템당 상위 K 이웃만 유지 (희소화)
MAX_MEMBERS_PER_META  = 2000  # 메타노드(도메인/장르) 멤버가 너무 클 때 샘플링 상한 (메모리/시간 보호)

ENCODINGS = ["utf-8-sig","utf-8","cp949","euc-kr","latin1"]
def read_csv_retry(path, **kwargs):
    last = None
    for enc in ENCODINGS:
        try:
            return pd.read_csv(path, encoding=enc, **kwargs)
        except Exception as e:
            last = e
    raise last

def main():
    # 1) 로드
    nodes = read_csv_retry(PATH_NODES)
    rawg  = read_csv_retry(PATH_RAWGEN) if os.path.exists(PATH_RAWGEN) else pd.DataFrame(columns=["content_id","source","raw_genre"])

    if nodes.empty:
        raise RuntimeError("graph_nodes.csv 비어있음")
    if "content_id" not in nodes.columns or "domain" not in nodes.columns:
        raise RuntimeError("graph_nodes.csv에 content_id/domain 필요")

    nodes = nodes.dropna(subset=["content_id","domain"]).copy()
    nodes["content_id"] = pd.to_numeric(nodes["content_id"], errors="coerce").astype("Int64")
    nodes = nodes[nodes["content_id"].notna()]
    nodes["content_id"] = nodes["content_id"].astype(int)

    # 도메인 맵
    domain_of = dict(nodes[["content_id","domain"]].values)
    all_items = set(domain_of.keys())
    N_items   = max(1, len(all_items))

    # 2) 메타노드 멤버 구성
    # (a) 도메인 메타노드: DOM:<domain>
    members_of_meta = defaultdict(list)
    for cid, dom in domain_of.items():
        members_of_meta[f"DOM:{dom}"].append(cid)

    # (b) '도메인별 원본 장르' 메타노드: GEN:<source>:<raw_genre>
    if not rawg.empty:
        rawg = rawg.dropna(subset=["content_id","source","raw_genre"]).copy()
        rawg["content_id"] = pd.to_numeric(rawg["content_id"], errors="coerce").astype("Int64")
        rawg = rawg[rawg["content_id"].notna()]
        rawg["content_id"] = rawg["content_id"].astype(int)
        # 같은 도메인에서만 의미를 주려면, 아이템의 도메인을 확인해서 묶기만 하면 됨.
        # 여기선 메타노드 ID에 도메인까지 넣어 '도메인별 장르'로 분리한다.
        # 예) GEN:tmdb:공포@AV, GEN:steam:Action@GAME
        for cid, src, g in rawg[["content_id","source","raw_genre"]].drop_duplicates().itertuples(index=False):
            dom = domain_of.get(cid)
            if not dom: 
                continue
            meta_id = f"GEN:{src}:{g}@{dom}"
            members_of_meta[meta_id].append(cid)

    # 3) 투영(아이템↔아이템): 메타노드 공동소속 기반 가중치
    def meta_contrib(meta_id, deg):
        # deg = 메타노드 멤버 수
        base = DOMAIN_BASE_WEIGHT if meta_id.startswith("DOM:") else GENRE_BASE_WEIGHT
        if deg <= 1:
            return 0.0
        # 허브 완화: 1 / (deg^ALPHA)
        w = base / (deg ** ALPHA)
        # 정보량(IDF) 보정: log(1 + N / deg)
        if USE_IDF:
            w *= math.log(1.0 + (N_items / float(deg)))
        return w

    edges = defaultdict(float)

    for meta_id, members in members_of_meta.items():
        members = list(set(members))
        deg = len(members)
        if deg <= 1:
            continue

        # 너무 큰 메타노드(예: 특정 도메인 전체)가 폭발하지 않게 샘플링/상한
        if deg > MAX_MEMBERS_PER_META:
            # 균일 샘플링 (재현성은 보장X. 필요시 random.seed() 추가)
            members = members[:MAX_MEMBERS_PER_META]
            deg = len(members)

        contrib = meta_contrib(meta_id, deg)
        if contrib == 0.0:
            continue

        # 근린 제한 방식: 각 멤버마다 앞쪽 K개와만 연결 (전쌍 O(n^2) 방지)
        K = min(TOPK_PER_ITEM, deg - 1)
        members.sort()
        for i, a in enumerate(members):
            # 도메인 다른 쌍은 제외 (도메인 일치 강제) — 도메인별 장르 메타는 이미 분리되어 있지만,
            # 도메인 메타에서도 안전하게 동일 도메인만 유지
            for b in members[i+1 : i+1+K]:
                if domain_of.get(a) != domain_of.get(b):
                    continue
                if a > b:
                    a, b = b, a
                edges[(a,b)] += contrib

    # 4) 아이템당 TOPK 이웃만 유지(양방향 기준 최대 가중치 유지)
    nbrs = defaultdict(list)
    for (a,b), w in edges.items():
        nbrs[a].append((b,w))
        nbrs[b].append((a,w))

    pruned = {}
    for a, lst in nbrs.items():
        lst.sort(key=lambda x: x[1], reverse=True)
        for b, w in lst[:TOPK_PER_ITEM]:
            key = (a,b) if a < b else (b,a)
            if key not in pruned or pruned[key] < w:
                pruned[key] = w

    df_edges = pd.DataFrame(
        [(a,b,w) for (a,b), w in pruned.items()],
        columns=["src_content_id","dst_content_id","weight"]
    )
    df_edges.to_csv(OUT_EDGES, index=False, encoding="utf-8-sig")
    print(f"✅ 저장: {OUT_EDGES} (edges={len(df_edges):,}, items={len(all_items):,})")

if __name__ == "__main__":
    main()


✅ 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\graph_edges_item_item.csv (edges=3,531, items=139)


# 아이템-아이템 가중 그래프 graphSAGE

In [2]:
# -*- coding: utf-8 -*-

# 아이템-아이템 가중 그래프 -> GraphSAGE 임베딩 (PyTorch Geometric)
# 입력 : C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\graph_edges_item_item.csv
#        (columns: src_content_id, dst_content_id, weight)
# 출력 : C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\item_embeddings_torch.csv
# 사용 : python learn_item_embeddings_sage.py

import os
import numpy as np
import pandas as pd
import torch
from torch import nn
from typing import Dict, Tuple
from torch_geometric.data import Data
from torch_geometric.nn import SAGEConv
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.utils import to_undirected, negative_sampling

# ===== 경로/하이퍼 =====
BASE = r"C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean"
EDGES_CSV = os.path.join(BASE, "graph_edges_item_item.csv")
OUT_CSV   = os.path.join(BASE, "item_embeddings_torch.csv")  # downstream과 호환

SEED         = 42
EMB_DIM      = 128      # 최종 임베딩 차원
HIDDEN       = 256
LAYERS       = 2        # SAGE 레이어 수
DROPOUT      = 0.1
LR           = 1e-3
EPOCHS       = 20
NEG_MULT     = 5        # 미니배치 음성 엣지 비율
USE_EDGE_W   = True     # 엣지 weight 반영(BCE 가중치)
DEVICE       = "cuda" if torch.cuda.is_available() else "cpu"

torch.manual_seed(SEED); np.random.seed(SEED)

# ---------- CSV 로딩 ----------
def read_edges(path: str) -> pd.DataFrame:
    encs = ["utf-8-sig","utf-8","cp949","euc-kr","latin1"]
    last=None
    for enc in encs:
        try:
            df = pd.read_csv(path, encoding=enc)
            break
        except Exception as e:
            last=e
    else:
        raise last

    need = {"src_content_id","dst_content_id"}
    if not need.issubset(df.columns):
        raise ValueError(f"{path}에 {need} 컬럼이 필요합니다.")

    # 정리
    for c in ["src_content_id","dst_content_id"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df = df.dropna(subset=["src_content_id","dst_content_id"]).copy()
    df["src_content_id"] = df["src_content_id"].astype(int)
    df["dst_content_id"] = df["dst_content_id"].astype(int)
    df = df[df["src_content_id"] != df["dst_content_id"]]
    df = df.drop_duplicates(subset=["src_content_id","dst_content_id"])

    # weight
    if "weight" in df.columns:
        df["weight"] = pd.to_numeric(df["weight"], errors="coerce").fillna(1.0).astype(float)
    else:
        df["weight"] = 1.0
    return df

def build_index(df: pd.DataFrame):
    nodes = sorted(set(df["src_content_id"]) | set(df["dst_content_id"]))
    id2idx = {nid:i for i,nid in enumerate(nodes)}
    idx2id = {i:nid for nid,i in id2idx.items()}
    idx_df = df.replace({"src_content_id": id2idx, "dst_content_id": id2idx})
    return idx_df, id2idx, idx2id

# ---------- 모델 ----------
class GraphSAGEEncoder(nn.Module):
    def __init__(self, num_nodes: int, in_dim: int, hidden: int, out_dim: int, layers: int, dropout: float):
        super().__init__()
        # 초기 노드 피처가 없으므로 학습 가능 임베딩을 입력으로 사용
        self.node_emb = nn.Embedding(num_nodes, in_dim)
        nn.init.xavier_uniform_(self.node_emb.weight)

        convs = []
        dims = [in_dim] + [hidden]*(layers-1) + [out_dim]
        for li in range(layers):
            convs.append(SAGEConv(dims[li], dims[li+1]))
        self.convs = nn.ModuleList(convs)
        self.act = nn.ReLU()
        self.drop = nn.Dropout(dropout)

    def forward(self, edge_index: torch.Tensor):
        x = self.node_emb.weight  # [N, in_dim]
        for li, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if li < len(self.convs)-1:
                x = self.act(x)
                x = self.drop(x)
        return x  # [N, out_dim]

def bce_link_loss(z: torch.Tensor,
                  pos_ei: torch.Tensor,
                  neg_ei: torch.Tensor,
                  pos_w: torch.Tensor = None):
    # 점곱 로짓
    pos_logits = (z[pos_ei[0]] * z[pos_ei[1]]).sum(dim=-1)      # [P]
    neg_logits = (z[neg_ei[0]] * z[neg_ei[1]]).sum(dim=-1)      # [N]

    logits = torch.cat([pos_logits, neg_logits], dim=0)
    labels = torch.cat([torch.ones_like(pos_logits),
                        torch.zeros_like(neg_logits)], dim=0)

    if pos_w is not None:
        # positive에만 가중(정규화)
        pos_w = pos_w / (pos_w.mean() + 1e-12)
        w = torch.cat([pos_w, torch.ones_like(neg_logits)], dim=0)
        return nn.functional.binary_cross_entropy_with_logits(logits, labels, weight=w)
    else:
        return nn.functional.binary_cross_entropy_with_logits(logits, labels)

# --- 호환 헬퍼: 학습용 양성 엣지 가져오기 ---
def get_pos_edge_label_index(d: Data) -> torch.Tensor:
    # 구버전 호환
    if hasattr(d, 'pos_edge_label_index'):
        return d.pos_edge_label_index
    # 최신: edge_label_index(+ edge_label)
    if hasattr(d, 'edge_label_index'):
        if hasattr(d, 'edge_label') and d.edge_label is not None:
            mask = (d.edge_label == 1)
            if mask.dim() > 0:
                return d.edge_label_index[:, mask]
        return d.edge_label_index
    # 최후 수단: 현재 그래프의 edge_index 사용
    return d.edge_index

def train(model, data, train_pos_ei, pos_weights=None):
    model = model.to(DEVICE)
    edge_index = data.edge_index.to(DEVICE)
    train_pos_ei = train_pos_ei.to(DEVICE)
    if pos_weights is not None:
        pos_weights = pos_weights.to(DEVICE)

    opt = torch.optim.Adam(model.parameters(), lr=LR)

    for ep in range(1, EPOCHS+1):
        model.train()
        opt.zero_grad()

        z = model(edge_index)   # [N, D]

        # negative sampling: positive 수 * NEG_MULT
        neg_ei = negative_sampling(
            edge_index=train_pos_ei,
            num_nodes=z.size(0),
            num_neg_samples=train_pos_ei.size(1) * NEG_MULT,
            method='sparse'
        )

        loss = bce_link_loss(z, train_pos_ei, neg_ei, pos_weights if USE_EDGE_W else None)
        loss.backward()
        opt.step()

        with torch.no_grad():
            zn = nn.functional.normalize(z, p=2, dim=1)
            pos_cos = (zn[train_pos_ei[0]] * zn[train_pos_ei[1]]).sum(dim=1).mean().item()
            neg_cos = (zn[neg_ei[0]] * zn[neg_ei[1]]).sum(dim=1).mean().item()
        print(f"[{ep:02d}/{EPOCHS}] loss={loss.item():.4f}  cos(pos)≈{pos_cos:.3f}  cos(neg)≈{neg_cos:.3f}")

    return model

def main():
    print("📥 엣지 로딩...")
    df = read_edges(EDGES_CSV)
    df_idx, id2idx, idx2id = build_index(df)
    num_nodes = len(id2idx)
    print(f"nodes={num_nodes:,}, edges={len(df_idx):,}")

    # PyG Data
    edge_index = torch.as_tensor(
        df_idx[["src_content_id","dst_content_id"]].to_numpy().T, dtype=torch.long
    )
    edge_index = to_undirected(edge_index, num_nodes=num_nodes)
    data = Data(num_nodes=num_nodes, edge_index=edge_index)

    # positive edge weight 맵(옵션)
    w_map: Dict[Tuple[int,int], float] = {}
    for a, b, w in df_idx[["src_content_id","dst_content_id","weight"]].itertuples(index=False):
        if a > b: a, b = b, a
        w_map[(a,b)] = float(w)

    # 링크 분할 (학습용 positive 인덱스만 사용)
    splitter = RandomLinkSplit(is_undirected=True, add_negative_train_samples=False, neg_sampling_ratio=0.0)
    train_data, _, _ = splitter(data)

    # ⬇️ 호환 헬퍼 사용
    train_pos = get_pos_edge_label_index(train_data)  # [2, P]

    pos_w = None
    if USE_EDGE_W:
        a = train_pos[0].cpu().numpy()
        b = train_pos[1].cpu().numpy()
        kk = []
        for x, y in zip(a, b):
            if x > y: x, y = y, x
            kk.append(w_map.get((int(x), int(y)), 1.0))
        pos_w = torch.tensor(kk, dtype=torch.float32)

    print("🧠 GraphSAGE 학습...")
    model = GraphSAGEEncoder(
        num_nodes=num_nodes,
        in_dim=EMB_DIM, hidden=HIDDEN, out_dim=EMB_DIM,
        layers=LAYERS, dropout=DROPOUT
    )
    model = train(model, data, train_pos, pos_w)

    print("💾 임베딩 저장...")
    model.eval()
    with torch.no_grad():
        z = model(data.edge_index.to(DEVICE)).detach().cpu()
        z = nn.functional.normalize(z, p=2, dim=1)

    rows = []
    for idx in range(num_nodes):
        cid = idx2id[idx]
        rows.append([cid] + list(map(float, z[idx].numpy().tolist())))
    cols = ["content_id"] + [f"emb_{i}" for i in range(EMB_DIM)]
    out = pd.DataFrame(rows, columns=cols).sort_values("content_id")
    out.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
    print(f"✅ 임베딩 저장: {OUT_CSV} (nodes={len(out):,}, dim={EMB_DIM})")

if __name__ == "__main__":
    main()


📥 엣지 로딩...
nodes=139, edges=3,531
🧠 GraphSAGE 학습...
[01/20] loss=0.7974  cos(pos)≈0.845  cos(neg)≈0.837
[02/20] loss=0.7441  cos(pos)≈0.741  cos(neg)≈0.727
[03/20] loss=0.7168  cos(pos)≈0.582  cos(neg)≈0.559
[04/20] loss=0.7049  cos(pos)≈0.421  cos(neg)≈0.394
[05/20] loss=0.7009  cos(pos)≈0.336  cos(neg)≈0.303
[06/20] loss=0.7004  cos(pos)≈0.330  cos(neg)≈0.290
[07/20] loss=0.7011  cos(pos)≈0.355  cos(neg)≈0.312
[08/20] loss=0.7016  cos(pos)≈0.378  cos(neg)≈0.327
[09/20] loss=0.7022  cos(pos)≈0.403  cos(neg)≈0.345
[10/20] loss=0.7019  cos(pos)≈0.410  cos(neg)≈0.342
[11/20] loss=0.7010  cos(pos)≈0.414  cos(neg)≈0.325
[12/20] loss=0.6998  cos(pos)≈0.389  cos(neg)≈0.296
[13/20] loss=0.6982  cos(pos)≈0.363  cos(neg)≈0.255
[14/20] loss=0.6969  cos(pos)≈0.347  cos(neg)≈0.216
[15/20] loss=0.6955  cos(pos)≈0.319  cos(neg)≈0.167
[16/20] loss=0.6947  cos(pos)≈0.315  cos(neg)≈0.136
[17/20] loss=0.6938  cos(pos)≈0.317  cos(neg)≈0.100
[18/20] loss=0.6932  cos(pos)≈0.309  cos(neg)≈0.079
[19/20] loss

# Noce2vec 아이템 임베딩

In [1]:
# learn_item_embeddings.py
# -*- coding: utf-8 -*-

#아이템-아이템 가중 그래프 -> Node2Vec 스타일 임베딩 (Pure PyTorch, gensim/scipy 불필요)
#입력 : C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\graph_edges_item_item.csv
#출력 : C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\item_embeddings_torch.csv
#사용 : python learn_item_embeddings.py
#필요 : pip install torch pandas numpy
import os, math, random
import numpy as np
import pandas as pd
import torch
from collections import defaultdict

# ===== 경로 =====
BASE = r"C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean"
EDGES_PATH = os.path.join(BASE, "graph_edges_item_item.csv")
OUT_EMB_CSV = os.path.join(BASE, "item_embeddings_torch.csv")

# ===== 하이퍼파라미터 =====
DIM            = 64       # 임베딩 차원
WALK_LENGTH    = 40       # 랜덤워크 길이
NUM_WALKS      = 10       # 노드당 워크 수
P_RETURN       = 1.0      # node2vec p (되돌아가기 선호 p↓)
Q_INOUT        = 1.0      # node2vec q (원거리 탐색 선호 q↑)
WINDOW         = 5        # Skip-gram 윈도우
NEGATIVE_K     = 5        # 네거티브 샘플 수
EPOCHS         = 3
BATCH_SIZE     = 8192
LR             = 0.025
SEED           = 42
DEVICE         = "cuda" if torch.cuda.is_available() else "cpu"

random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

# ---------- 그래프 로드 & 가중 룰렛 준비 ----------
def load_graph(edges_csv):
    df = pd.read_csv(edges_csv, encoding="utf-8-sig")
    need = {"src_content_id","dst_content_id","weight"}
    if not need.issubset(df.columns):
        raise ValueError(f"{edges_csv}에 {need} 컬럼이 필요합니다.")

    adj = defaultdict(list)   # node -> [(nbr, w), ...]
    nodes = set()
    for a, b, w in df[["src_content_id","dst_content_id","weight"]].itertuples(index=False):
        if a == b: 
            continue
        w = float(w) if pd.notna(w) else 1.0
        adj[a].append((b, w))
        adj[b].append((a, w))
        nodes.add(a); nodes.add(b)

    # 누적확률(룰렛휠) 사전
    cumsums = {}
    for u, lst in adj.items():
        if not lst:
            continue
        nbrs, ws = zip(*lst)
        ws = np.maximum(np.array(ws, dtype=float), 1e-12)
        csum = np.cumsum(ws); cumsums[u] = (np.array(nbrs, dtype=int), csum / csum[-1])
    return adj, cumsums, sorted(nodes)

def weighted_choice(nbrs, cprobs):
    r = random.random()
    lo, hi = 0, len(cprobs)-1
    while lo < hi:
        mid = (lo + hi) // 2
        if cprobs[mid] < r: lo = mid + 1
        else: hi = mid
    return int(nbrs[lo])

# ---------- Node2Vec 가중 랜덤워크 ----------
def node2vec_walks(adj, cumsums, nodes, walk_length, num_walks, p, q):
    walks = []
    base_nodes = list(nodes)
    for _ in range(num_walks):
        random.shuffle(base_nodes)
        for start in base_nodes:
            if start not in cumsums:
                continue
            walk = [start]
            if walk_length == 1:
                walks.append(walk); continue

            # 첫 스텝: 가중치 비례 선택
            nbrs, cprobs = cumsums[start]
            if len(nbrs) == 0:
                walks.append(walk); continue
            curr = weighted_choice(nbrs, cprobs)
            walk.append(curr); prev = start

            for _ in range(2, walk_length):
                cand = adj.get(curr, [])
                if not cand: break
                cand_nodes = [n for n,_ in cand]
                cand_w = []
                prev_nbrs = {n for n,_ in adj.get(prev, [])}
                for nxt, w in cand:
                    # p/q 바이어스(근사)
                    if nxt == prev: bias = 1.0 / p
                    elif nxt in prev_nbrs: bias = 1.0
                    else: bias = 1.0 / q
                    cand_w.append(max(w,1e-12) * bias)
                cw = np.array(cand_w, dtype=float)
                cs = np.cumsum(cw); cs /= cs[-1]
                idx = np.searchsorted(cs, random.random())
                nxt = cand_nodes[min(idx, len(cand_nodes)-1)]
                walk.append(nxt); prev, curr = curr, nxt
            walks.append(walk)
    return walks

# ---------- Skip-gram(네거티브 샘플링) ----------
def generate_pairs(walks, window):
    pairs = []
    for walk in walks:
        L = len(walk)
        for i in range(L):
            c = walk[i]
            l = max(0, i-window); r = min(L, i+window+1)
            for j in range(l, r):
                if j == i: continue
                pairs.append((c, walk[j]))
    return pairs

class SkipGramNS(torch.nn.Module):
    def __init__(self, vocab_size, dim):
        super().__init__()
        self.in_emb  = torch.nn.Embedding(vocab_size, dim)
        self.out_emb = torch.nn.Embedding(vocab_size, dim)
        torch.nn.init.uniform_(self.in_emb.weight,  -0.5/dim, 0.5/dim)
        torch.nn.init.uniform_(self.out_emb.weight, -0.5/dim, 0.5/dim)

    def forward(self, center, pos, neg):
        # center:[B], pos:[B], neg:[B,K]
        v  = self.in_emb(center)           # [B, D]
        u  = self.out_emb(pos)             # [B, D]
        uv = (v * u).sum(dim=1)            # [B]
        pos_loss = torch.nn.functional.logsigmoid(uv).mean()

        neg_u = self.out_emb(neg)          # [B, K, D]
        neg_uv = torch.bmm(neg_u, v.unsqueeze(2)).squeeze(2)  # [B, K]
        neg_loss = torch.nn.functional.logsigmoid(-neg_uv).mean()
        return -(pos_loss + neg_loss)      # minimize

def train_skipgram_ns(pairs, id2idx, epochs=3, batch_size=8192, dim=64, neg_k=5, lr=0.025):
    # 노드 인덱싱
    vocab = sorted(id2idx.keys())
    vocab_size = len(vocab)

    # 빈도 기반 네거티브 샘플링 분포(0.75 승)
    counts = defaultdict(int)
    for a,b in pairs:
        counts[a] += 1; counts[b] += 1
    idx_counts = np.zeros(vocab_size, dtype=np.float64)
    for nid, cnt in counts.items():
        idx_counts[id2idx[nid]] = cnt
    prob = idx_counts ** 0.75
    prob = prob / prob.sum()
    alias_table = np.cumsum(prob)

    def sample_neg(B, K):
        # 누적분포 기반 벡터화 샘플링
        r = np.random.rand(B, K)
        idx = np.searchsorted(alias_table, r, side="right")
        return torch.from_numpy(idx.astype(np.int64))

    # 학습 데이터 텐서화(인덱스 변환)
    centers = torch.tensor([id2idx[a] for a,_ in pairs], dtype=torch.long)
    contexts= torch.tensor([id2idx[b] for _,b in pairs], dtype=torch.long)

    ds_size = len(pairs)
    model = SkipGramNS(vocab_size, dim).to(DEVICE)
    optim = torch.optim.SGD(model.parameters(), lr=lr)

    for epoch in range(1, epochs+1):
        perm = torch.randperm(ds_size)
        centers = centers[perm]; contexts = contexts[perm]
        total_loss = 0.0; steps = 0

        for i in range(0, ds_size, batch_size):
            c_batch = centers[i:i+batch_size].to(DEVICE)
            p_batch = contexts[i:i+batch_size].to(DEVICE)
            B = c_batch.size(0)
            n_batch = sample_neg(B, neg_k).to(DEVICE)

            loss = model(c_batch, p_batch, n_batch)
            optim.zero_grad(); loss.backward(); optim.step()

            total_loss += loss.item(); steps += 1

        avg = total_loss / max(1, steps)
        print(f"[Epoch {epoch}/{epochs}] loss={avg:.4f}")

    with torch.no_grad():
        emb = model.in_emb.weight.detach().cpu().numpy()
    return emb  # shape [vocab_size, dim]

def main():
    print("📥 그래프 로딩...")
    adj, cumsums, nodes = load_graph(EDGES_PATH)
    print(f"nodes={len(nodes):,}, edges(undirected)≈{sum(len(v) for v in adj.values())//2:,}")

    print("🚶 랜덤워크 생성...")
    walks = node2vec_walks(
        adj, cumsums, nodes,
        walk_length=WALK_LENGTH,
        num_walks=NUM_WALKS,
        p=P_RETURN, q=Q_INOUT
    )
    avg_len = np.mean([len(w) for w in walks]) if walks else 0
    print(f"walks={len(walks):,}, avg_len≈{avg_len:.1f}")

    print("🧩 학습 쌍 생성(Skip-gram window)...")
    pairs = generate_pairs(walks, WINDOW)
    print(f"pairs={len(pairs):,}")

    # 노드 id → 연속 index 매핑
    id2idx = {nid:i for i, nid in enumerate(sorted(nodes))}
    print("🧠 Skip-gram(NS) 학습 시작...")
    emb = train_skipgram_ns(
        pairs, id2idx,
        epochs=EPOCHS, batch_size=BATCH_SIZE,
        dim=DIM, neg_k=NEGATIVE_K, lr=LR
    )

    # CSV 저장
    idx2id = {i:nid for nid,i in id2idx.items()}
    rows = [[idx2id[i]] + list(map(float, emb[i])) for i in range(len(idx2id))]
    cols = ["content_id"] + [f"emb_{i}" for i in range(DIM)]
    out = pd.DataFrame(rows, columns=cols).sort_values("content_id")
    out.to_csv(OUT_EMB_CSV, index=False, encoding="utf-8-sig")
    print(f"✅ 임베딩 저장: {OUT_EMB_CSV} (nodes={len(out)}, dim={DIM})")

if __name__ == "__main__":
    main()


📥 그래프 로딩...
nodes=139, edges(undirected)≈3,531
🚶 랜덤워크 생성...
walks=1,390, avg_len≈40.0
🧩 학습 쌍 생성(Skip-gram window)...
pairs=514,300
🧠 Skip-gram(NS) 학습 시작...
[Epoch 1/3] loss=1.3863
[Epoch 2/3] loss=1.3863
[Epoch 3/3] loss=1.3863
✅ 임베딩 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\item_embeddings_torch.csv (nodes=139, dim=64)


# 유저 임베딩 생성

In [6]:
# -*- coding: utf-8 -*-
# 유저 임베딩 생성 (아이템 임베딩 + 선호 도메인/장르 centroid)

import os
import numpy as np
import pandas as pd
from typing import Optional, List

BASE = r"C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean"
ITEM_EMB_1 = os.path.join(BASE, "item_embeddings_torch.csv")
ITEM_EMB_2 = os.path.join(BASE, "item_embeddings.csv")
CONTENTS   = os.path.join(BASE, "contents.csv")
RAW_GENRES = os.path.join(BASE, "content_raw_genres.csv")
UPREF      = os.path.join(BASE, "user_preferences.csv")
UP_CTYPES  = os.path.join(BASE, "user_preferred_content_types.csv")
UP_GENRES  = os.path.join(BASE, "user_preferred_genres.csv")
OUT_USER   = os.path.join(BASE, "user_embeddings.csv")

def read_csv_retry(path, encodings=("utf-8-sig","utf-8","cp949","euc-kr","latin1"), **kw) -> Optional[pd.DataFrame]:
    if not os.path.exists(path):
        return None
    last = None
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, **kw)
        except Exception as e:
            last = e
    raise last

def load_item_embeddings() -> pd.DataFrame:
    # 명시적 선택 (DataFrame에 or 금지)
    df1 = read_csv_retry(ITEM_EMB_1)
    if df1 is not None and not df1.empty:
        df = df1
        src = os.path.basename(ITEM_EMB_1)
    else:
        df2 = read_csv_retry(ITEM_EMB_2)
        if df2 is not None and not df2.empty:
            df = df2
            src = os.path.basename(ITEM_EMB_2)
        else:
            raise RuntimeError("아이템 임베딩 CSV가 없습니다: item_embeddings_torch.csv 또는 item_embeddings.csv")

    if "content_id" not in df.columns:
        raise RuntimeError("아이템 임베딩 CSV에 content_id 컬럼이 필요합니다.")

    emb_cols = [c for c in df.columns if c.startswith("emb_")]
    if not emb_cols:
        raise RuntimeError("아이템 임베딩 CSV에 emb_로 시작하는 임베딩 컬럼이 없습니다.")

    df["content_id"] = pd.to_numeric(df["content_id"], errors="coerce").astype("Int64")
    df = df[df["content_id"].notna()].copy()
    df["content_id"] = df["content_id"].astype(int)

    for c in emb_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df.dropna(subset=emb_cols, inplace=True)

    # L2 normalize
    M = df[emb_cols].to_numpy(np.float32)
    M = M / (np.linalg.norm(M, axis=1, keepdims=True) + 1e-12)
    df.loc[:, emb_cols] = M

    print(f"✅ 아이템 임베딩 로드: {src} (items={len(df)}, dim={len(emb_cols)})")
    return df[["content_id"] + emb_cols]

# content_type → domain 매핑
CTYPE2DOMAIN = {
    "movie": "AV", "tv": "AV",
    "game": "GAME",
    "webnovel": "WEBNOVEL", "webtoon": "WEBNOVEL"
}

def main():
    emb = load_item_embeddings()
    emb_cols = [c for c in emb.columns if c.startswith("emb_")]
    M = emb[emb_cols].to_numpy(np.float32)
    ids = emb["content_id"].to_numpy(int)

    # contents.csv → domain
    contents = read_csv_retry(CONTENTS)
    if contents is None or contents.empty:
        raise RuntimeError("contents.csv 필요.")
    contents = contents[["content_id", "domain"]].copy()
    contents["content_id"] = pd.to_numeric(contents["content_id"], errors="coerce").astype("Int64")
    contents = contents[contents["content_id"].notna()].copy()
    contents["content_id"] = contents["content_id"].astype(int)
    emb_meta = emb.merge(contents, on="content_id", how="left")

    # domain centroids
    domain_vecs = {}
    dom_counts = {}
    for dom, grp in emb_meta.dropna(subset=["domain"]).groupby("domain"):
        mat = grp[emb_cols].to_numpy(np.float32)
        if len(mat) == 0: 
            continue
        v = mat.mean(axis=0)
        v = v / (np.linalg.norm(v) + 1e-12)
        domain_vecs[str(dom)] = v
        dom_counts[str(dom)] = len(mat)
    print(f"✅ 도메인 센트로이드: { {k: dom_counts[k] for k in sorted(dom_counts)} }")

    # genre centroids (optional)
    genre_vecs = {}
    if os.path.exists(RAW_GENRES):
        rawg = read_csv_retry(RAW_GENRES)
        if rawg is not None and not rawg.empty and {"content_id","raw_genre"}.issubset(rawg.columns):
            rawg = rawg[["content_id", "raw_genre"]].dropna().copy()
            rawg["content_id"] = pd.to_numeric(rawg["content_id"], errors="coerce").astype("Int64")
            rawg = rawg[rawg["content_id"].notna()].copy()
            rawg["content_id"] = rawg["content_id"].astype(int)
            rawg["g"] = rawg["raw_genre"].astype(str).str.strip().str.lower()

            idx_map = {cid: i for i, cid in enumerate(ids)}
            for g, grp in rawg.groupby("g"):
                idxs = [idx_map[c] for c in grp["content_id"].tolist() if c in idx_map]
                if not idxs:
                    continue
                v = M[idxs].mean(axis=0)
                v = v / (np.linalg.norm(v) + 1e-12)
                genre_vecs[g] = v
            print(f"✅ 장르 센트로이드 수: {len(genre_vecs)}")
        else:
            print("ℹ️ content_raw_genres.csv 비어있음 또는 컬럼 불충분 → 장르 센트로이드 생략")
    else:
        print("ℹ️ content_raw_genres.csv 없음 → 장르 센트로이드 생략")

    # user prefs
    upref = read_csv_retry(UPREF)
    uctype = read_csv_retry(UP_CTYPES)
    ugen = read_csv_retry(UP_GENRES)
    if upref is None or upref.empty:
        raise RuntimeError("user_preferences.csv 필요.")

    ctype_map = {}
    if uctype is not None and not uctype.empty and {"preference_id", "content_type"}.issubset(uctype.columns):
        for pid, grp in uctype.groupby("preference_id"):
            ctype_map[int(pid)] = [str(x).strip() for x in grp["content_type"].tolist()]

    genre_map = {}
    if ugen is not None and not ugen.empty and {"preference_id", "genre"}.issubset(ugen.columns):
        for pid, grp in ugen.groupby("preference_id"):
            genre_map[int(pid)] = [str(x).strip().lower() for x in grp["genre"].tolist()]

    def ctypes_to_domains(cts: List[str]) -> List[str]:
        outs = set()
        for ct in cts:
            d = CTYPE2DOMAIN.get(str(ct).lower())
            if d:
                outs.add(d)
        return sorted(outs)

    # 가중치
    ALPHA_DOM, ALPHA_GEN = 0.6, 0.4

    rows = []
    skipped = 0
    for _, r in upref.iterrows():
        if pd.isna(r.get("id")):
            skipped += 1
            continue
        uid = int(r["id"])
        uname = r.get("username", "")

        # domains
        doms = ctypes_to_domains(ctype_map.get(uid, []))
        dom_vecs = [domain_vecs[d] for d in doms if d in domain_vecs]
        v_dom = None
        if dom_vecs:
            v_dom = np.mean(np.stack(dom_vecs, axis=0), axis=0)
            v_dom = v_dom / (np.linalg.norm(v_dom) + 1e-12)

        # genres
        gnames = [g for g in (genre_map.get(uid, [])) if g in genre_vecs]
        g_vecs = [genre_vecs[g] for g in gnames]
        v_gen = None
        if g_vecs:
            v_gen = np.mean(np.stack(g_vecs, axis=0), axis=0)
            v_gen = v_gen / (np.linalg.norm(v_gen) + 1e-12)

        # user vector
        if v_dom is None and v_gen is None:
            # 폴백: 전체 아이템 평균
            v_user = M.mean(axis=0)
        elif v_dom is None:
            v_user = v_gen
        elif v_gen is None:
            v_user = v_dom
        else:
            v_user = ALPHA_DOM * v_dom + ALPHA_GEN * v_gen
        v_user = v_user / (np.linalg.norm(v_user) + 1e-12)

        rows.append([uid, uname] + list(map(float, v_user)))

    if not rows:
        raise RuntimeError("유저 임베딩 생성 결과가 비었습니다. 입력 파일/선호 정보 확인 필요.")

    dim = M.shape[1]
    out = pd.DataFrame(rows, columns=["user_id", "username"] + [f"emb_{i}" for i in range(dim)])
    out.sort_values("user_id").to_csv(OUT_USER, index=False, encoding="utf-8-sig")
    print(f"✅ user_embeddings.csv 저장: {OUT_USER} (users={len(out)}, skipped_rows={skipped})")

if __name__ == "__main__":
    main()


✅ 아이템 임베딩 로드: item_embeddings_torch.csv (items=139, dim=64)
✅ 도메인 센트로이드: {'AV': 60, 'GAME': 24, 'WEBNOVEL': 55}
✅ 장르 센트로이드 수: 8
✅ user_embeddings.csv 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\user_embeddings.csv (users=10, skipped_rows=0)


# 유저 아이템 임베딩 top-k추천

In [8]:
# -*- coding: utf-8 -*-

#유저/아이템 임베딩만으로 Top-K 추천 (contents.csv 불필요)
#입력:
#  clean/user_embeddings.csv                  (user_id, username, emb_0..)
#  clean/item_embeddings_torch.csv 또는 item_embeddings.csv (content_id, emb_0..)
#출력:
#  clean/recommendations_topK.csv             (user_id, username, rank, content_id, score)


import os
import re
import numpy as np
import pandas as pd
from typing import Optional

BASE = r"C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean"
USER_EMB   = os.path.join(BASE, "user_embeddings.csv")
ITEM_EMB_1 = os.path.join(BASE, "item_embeddings_torch.csv")
ITEM_EMB_2 = os.path.join(BASE, "item_embeddings.csv")
OUT_RECS   = os.path.join(BASE, "recommendations_topK.csv")

TOPK = 5
EXCLUDE_DUP_CONTENTS = True   # content_id 중복행이 있으면 첫 번째만 사용
ASSUME_NORMALIZED = True      # 임베딩이 이미 L2 정규화되어 있다면 True, 아니면 False로

def read_csv_retry(path, encodings=("utf-8-sig","utf-8","cp949","euc-kr","latin1"), **kw) -> Optional[pd.DataFrame]:
    if not os.path.exists(path):
        return None
    last=None
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, **kw)
        except Exception as e:
            last=e
    raise last

def l2norm(X: np.ndarray) -> np.ndarray:
    return X / (np.linalg.norm(X, axis=1, keepdims=True) + 1e-12)

def _coerce_content_id_column(df: pd.DataFrame) -> pd.Series:
    """여러 이름/형태로 저장된 content_id를 최대한 복구."""
    df = df.copy()
    orig_cols = list(df.columns)
    df.columns = [c.strip() for c in df.columns]
    name_map = {c.lower(): c for c in df.columns}
    for key in ["content_id", "contentid", "content id", "id", "unnamed: 0"]:
        if key in name_map:
            cand = name_map[key]
            s = df[cand]
            if s.dtype == object:
                s = s.astype(str).str.strip()
            s = pd.to_numeric(s, errors="coerce").astype("Int64")
            if s.isna().all():
                continue
            return s
    raise RuntimeError(f"content_id 열을 찾지 못했습니다. 원본 컬럼: {orig_cols}")

def load_user_embeddings() -> pd.DataFrame:
    u = read_csv_retry(USER_EMB)
    if u is None or u.empty:
        raise RuntimeError("user_embeddings.csv 필요.")
    u = u.copy()
    u.columns = [c.strip() for c in u.columns]
    if "user_id" not in u.columns:
        # 혹시 id로 저장되었을 수도 있음
        if "id" in u.columns:
            u = u.rename(columns={"id": "user_id"})
        else:
            raise RuntimeError("user_embeddings.csv에 user_id 컬럼 필요.")
    u["user_id"] = pd.to_numeric(u["user_id"], errors="coerce").astype("Int64")
    u = u[u["user_id"].notna()].copy()
    u["user_id"] = u["user_id"].astype(int)

    emb_cols = [c for c in u.columns if c.lower().startswith("emb_")]
    if not emb_cols:
        raise RuntimeError("user_embeddings.csv에 emb_로 시작하는 임베딩 컬럼이 없습니다.")
    for c in emb_cols:
        u[c] = pd.to_numeric(u[c], errors="coerce")
    u.dropna(subset=emb_cols, inplace=True)
    if not ASSUME_NORMALIZED:
        u.loc[:, emb_cols] = l2norm(u[emb_cols].to_numpy(np.float32))
    return u[["user_id", u.columns[u.columns.str.lower()=="username"][0] if "username" in [x.lower() for x in u.columns] else None] + emb_cols].rename(columns={None:"username"})

def load_item_embeddings() -> pd.DataFrame:
    i1 = read_csv_retry(ITEM_EMB_1)
    if i1 is not None and not i1.empty:
        df = i1; src = os.path.basename(ITEM_EMB_1)
    else:
        i2 = read_csv_retry(ITEM_EMB_2)
        if i2 is None or i2.empty:
            raise RuntimeError("item_embeddings CSV 필요: item_embeddings_torch.csv 또는 item_embeddings.csv")
        df = i2; src = os.path.basename(ITEM_EMB_2)

    df = df.copy()
    df.columns = [c.strip() for c in df.columns]
    df["content_id"] = _coerce_content_id_column(df)

    emb_cols = [c for c in df.columns if c.lower().startswith("emb_")]
    if not emb_cols:
        # 혹시 다른 접두어로 저장되었는지 후보 보여주기
        alt = [c for c in df.columns if re.search(r"\d+$", c)]
        raise RuntimeError(f"임베딩 컬럼(emb_*) 없음. 확인 필요. 후보: {alt}")

    for c in emb_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df = df.dropna(subset=emb_cols + ["content_id"]).copy()
    df["content_id"] = df["content_id"].astype(int)

    if EXCLUDE_DUP_CONTENTS:
        df = df.drop_duplicates(subset=["content_id"], keep="first").reset_index(drop=True)

    if not ASSUME_NORMALIZED:
        df.loc[:, emb_cols] = l2norm(df[emb_cols].to_numpy(np.float32))

    print(f"✅ 아이템 임베딩 로드: {src} (items={len(df)}, dim={len(emb_cols)})")
    return df[["content_id"] + emb_cols]

def main():
    u = load_user_embeddings()
    i = load_item_embeddings()

    ucols = [c for c in u.columns if c.lower().startswith("emb_")]
    icols = [c for c in i.columns if c.lower().startswith("emb_")]

    U = u[ucols].to_numpy(np.float32)
    I = i[icols].to_numpy(np.float32)
    # 혹시 ASSUME_NORMALIZED=True인데 실제로 정규화 안되어 있으면, 코사인 = 내적이 아닐 수 있으니 보정
    # 안전하게 한 번 더 정규화
    U = l2norm(U)
    I = l2norm(I)

    item_ids = i["content_id"].to_numpy(int)
    user_ids = u["user_id"].to_numpy(int)
    usernames = u["username"].astype(str).to_numpy() if "username" in u.columns else np.array([""]*len(u))

    rec_rows = []
    for r in range(U.shape[0]):
        sims = (I @ U[r:r+1].T).reshape(-1)   # 코사인(정규화 가정)
        k = min(TOPK, len(sims))
        if k == 0: 
            continue
        part = np.argpartition(-sims, k-1)[:k]
        order = part[np.argsort(-sims[part])]
        top_ids = item_ids[order]
        top_scs = sims[order]
        uid = int(user_ids[r]); uname = usernames[r] if r < len(usernames) else ""
        for rank, (cid, sc) in enumerate(zip(top_ids, top_scs), start=1):
            rec_rows.append([uid, uname, rank, int(cid), float(sc)])

    rec = pd.DataFrame(rec_rows, columns=["user_id","username","rank","content_id","score"])
    rec.sort_values(["user_id","rank"]).to_csv(OUT_RECS, index=False, encoding="utf-8-sig")
    print(f"✅ recommendations_topK.csv 저장: {OUT_RECS} (rows={len(rec)})")

if __name__ == "__main__":
    main()


✅ 아이템 임베딩 로드: item_embeddings_torch.csv (items=139, dim=64)
✅ recommendations_topK.csv 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\recommendations_topK.csv (rows=50)


# 도메인 별 추천 아이템 생성

In [9]:
# -*- coding: utf-8 -*-

#도메인별 Top-N 추천 (유저/아이템 임베딩 + contents.csv)
#입력:
#  clean/user_embeddings.csv                  (user_id, username, emb_0..)
#  clean/item_embeddings_torch.csv 또는 item_embeddings.csv (content_id, emb_0..)
#  clean/contents.csv                         (content_id, domain)
#출력:
#  clean/recommendations_per_domain.csv       (user_id, username, rank, content_id, score, domain)

#옵션:
#  DOMAIN_QUOTA: 각 도메인별 추천 개수 (예: {'AV':3,'GAME':3,'WEBNOVEL':3})
#  FILL_WITH_GLOBAL: 도메인 내 후보 부족 시 전체에서 채움

import os, re
import numpy as np
import pandas as pd
from typing import Optional, Dict

# ===== 경로 =====
BASE = r"C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean"
USER_EMB   = os.path.join(BASE, "user_embeddings.csv")
ITEM_EMB_1 = os.path.join(BASE, "item_embeddings_torch.csv")
ITEM_EMB_2 = os.path.join(BASE, "item_embeddings.csv")
CONTENTS   = os.path.join(BASE, "contents.csv")
OUT_CSV    = os.path.join(BASE, "recommendations_per_domain.csv")

# ===== 설정 =====
DOMAIN_QUOTA: Dict[str, int] = {"AV": 3, "GAME": 3, "WEBNOVEL": 3}   # 도메인별 개수
FILL_WITH_GLOBAL = True    # 도메인 후보 부족 시 글로벌 Top으로 보충
ASSUME_NORMALIZED = True   # 임베딩이 이미 L2 정규화라면 True
EXCLUDE_DUP_CONTENTS = True

# ===== 유틸 =====
def read_csv_retry(path, encodings=("utf-8-sig","utf-8","cp949","euc-kr","latin1"), **kw) -> Optional[pd.DataFrame]:
    if not os.path.exists(path):
        return None
    last=None
    for enc in encodings:
        try:
            return pd.read_csv(path, encoding=enc, **kw)
        except Exception as e:
            last=e
    raise last

def l2norm(X: np.ndarray) -> np.ndarray:
    return X / (np.linalg.norm(X, axis=1, keepdims=True) + 1e-12)

def _coerce_content_id_column(df: pd.DataFrame) -> pd.Series:
    df = df.copy()
    orig_cols = list(df.columns)
    df.columns = [c.strip() for c in df.columns]
    name_map = {c.lower(): c for c in df.columns}
    for key in ["content_id", "contentid", "content id", "id", "unnamed: 0"]:
        if key in name_map:
            s = df[name_map[key]]
            if s.dtype == object:
                s = s.astype(str).str.strip()
            s = pd.to_numeric(s, errors="coerce").astype("Int64")
            if not s.isna().all():
                return s
    raise RuntimeError(f"content_id 열을 찾지 못했습니다. 원본 컬럼: {orig_cols}")

def load_user_embeddings() -> pd.DataFrame:
    u = read_csv_retry(USER_EMB)
    if u is None or u.empty:
        raise RuntimeError("user_embeddings.csv 필요.")
    u = u.copy()
    u.columns = [c.strip() for c in u.columns]
    if "user_id" not in u.columns:
        if "id" in u.columns:
            u = u.rename(columns={"id": "user_id"})
        else:
            raise RuntimeError("user_embeddings.csv에 user_id 컬럼 필요.")
    u["user_id"] = pd.to_numeric(u["user_id"], errors="coerce").astype("Int64")
    u = u[u["user_id"].notna()].copy()
    u["user_id"] = u["user_id"].astype(int)

    emb_cols = [c for c in u.columns if c.lower().startswith("emb_")]
    if not emb_cols:
        raise RuntimeError("user_embeddings.csv에 emb_로 시작하는 임베딩 컬럼이 없습니다.")
    for c in emb_cols:
        u[c] = pd.to_numeric(u[c], errors="coerce")
    u.dropna(subset=emb_cols, inplace=True)
    if not ASSUME_NORMALIZED:
        u.loc[:, emb_cols] = l2norm(u[emb_cols].to_numpy(np.float32))

    # username 컬럼 유연 처리
    uname_col = None
    for c in u.columns:
        if c.lower() == "username":
            uname_col = c; break
    if uname_col is None:
        u["username"] = ""
        uname_col = "username"

    return u[["user_id", uname_col] + emb_cols].rename(columns={uname_col: "username"})

def load_item_embeddings() -> pd.DataFrame:
    i1 = read_csv_retry(ITEM_EMB_1)
    if i1 is not None and not i1.empty:
        df = i1; src = os.path.basename(ITEM_EMB_1)
    else:
        i2 = read_csv_retry(ITEM_EMB_2)
        if i2 is None or i2.empty:
            raise RuntimeError("item_embeddings CSV 필요: item_embeddings_torch.csv 또는 item_embeddings.csv")
        df = i2; src = os.path.basename(ITEM_EMB_2)

    df = df.copy()
    df.columns = [c.strip() for c in df.columns]
    df["content_id"] = _coerce_content_id_column(df)

    emb_cols = [c for c in df.columns if c.lower().startswith("emb_")]
    if not emb_cols:
        alt = [c for c in df.columns if re.search(r"\d+$", c)]
        raise RuntimeError(f"임베딩 컬럼(emb_*) 없음. 확인 필요. 후보: {alt}")

    for c in emb_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df = df.dropna(subset=emb_cols + ["content_id"]).copy()
    df["content_id"] = df["content_id"].astype(int)

    if EXCLUDE_DUP_CONTENTS:
        df = df.drop_duplicates(subset=["content_id"], keep="first").reset_index(drop=True)

    if not ASSUME_NORMALIZED:
        df.loc[:, emb_cols] = l2norm(df[emb_cols].to_numpy(np.float32))

    print(f"✅ 아이템 임베딩 로드: {src} (items={len(df)}, dim={len(emb_cols)})")
    return df[["content_id"] + emb_cols]

def main():
    # --- 데이터 로드 ---
    u = load_user_embeddings()
    i = load_item_embeddings()

    contents = read_csv_retry(CONTENTS)
    if contents is None or contents.empty or {"content_id","domain"} - set(contents.columns):
        raise RuntimeError("contents.csv 필요 (content_id, domain).")
    dom_map = contents[["content_id","domain"]].dropna().copy()
    dom_map["content_id"] = pd.to_numeric(dom_map["content_id"], errors="coerce").astype("Int64")
    dom_map = dom_map[dom_map["content_id"].notna()].copy()
    dom_map["content_id"] = dom_map["content_id"].astype(int)
    dom_map = dom_map.set_index("content_id")["domain"].to_dict()

    ucols = [c for c in u.columns if c.lower().startswith("emb_")]
    icols = [c for c in i.columns if c.lower().startswith("emb_")]

    U = u[ucols].to_numpy(np.float32); U = l2norm(U)   # 안전하게 한 번 더 정규화
    I = i[icols].to_numpy(np.float32); I = l2norm(I)
    item_ids = i["content_id"].to_numpy(int)

    # --- 전역 Top 계산에 사용할 전체 점수(유저별) 준비용 ---
    # (대규모면 유저별로 on-the-fly 계산해도 OK)
    rec_rows = []
    for r in range(U.shape[0]):
        uvec = U[r:r+1, :]                   # [1, D]
        sims = (I @ uvec.T).reshape(-1)      # [N,]

        # 1) 도메인별 quota 채우기
        picked_mask = np.zeros_like(sims, dtype=bool)
        per_domain_picks = []
        for dom, need in DOMAIN_QUOTA.items():
            if need <= 0: 
                continue
            # 해당 도메인 아이템 마스크
            dom_mask = np.array([dom_map.get(int(cid)) == dom for cid in item_ids], dtype=bool)
            if not dom_mask.any():
                continue
            # 아직 안 뽑힌 + 해당 도메인
            cand_mask = dom_mask & (~picked_mask)
            if not cand_mask.any():
                continue
            cand_idx = np.where(cand_mask)[0]
            cand_scores = sims[cand_idx]

            k = min(need, len(cand_idx))
            part = np.argpartition(-cand_scores, k-1)[:k]
            order = part[np.argsort(-cand_scores[part])]
            chosen = cand_idx[order]

            picked_mask[chosen] = True
            for j in chosen:
                per_domain_picks.append((int(item_ids[j]), float(sims[j]), dom))

        # 2) 부족분 글로벌 Top으로 보충(옵션)
        total_need = sum(max(0, n) for n in DOMAIN_QUOTA.values())
        if FILL_WITH_GLOBAL and picked_mask.sum() < total_need:
            remain = total_need - picked_mask.sum()
            left_idx = np.where(~picked_mask)[0]
            if len(left_idx) > 0:
                left_scores = sims[left_idx]
                k2 = min(remain, len(left_idx))
                part = np.argpartition(-left_scores, k2-1)[:k2]
                order = part[np.argsort(-left_scores[part])]
                chosen = left_idx[order]
                for j in chosen:
                    c = int(item_ids[j])
                    per_domain_picks.append((c, float(sims[j]), dom_map.get(c, "")))
                    picked_mask[j] = True

        # 3) rank 매기고 저장
        per_domain_picks.sort(key=lambda x: -x[1])
        uid = int(u.loc[r, "user_id"]); uname = str(u.loc[r, "username"])
        for rank, (cid, sc, dom) in enumerate(per_domain_picks, start=1):
            rec_rows.append([uid, uname, rank, cid, sc, dom])

    rec = pd.DataFrame(rec_rows, columns=["user_id","username","rank","content_id","score","domain"])
    rec.sort_values(["user_id","rank"]).to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
    print(f"✅ recommendations_per_domain.csv 저장: {OUT_CSV} (rows={len(rec)})")

    # 콘솔 요약
    try:
        for uid in sorted(rec["user_id"].unique()):
            sub = rec[rec["user_id"]==uid].sort_values("rank")
            print(f"\n[User {uid}] 총 {len(sub)}개")
            for _,row in sub.iterrows():
                print(f"  #{int(row['rank']):2d}  cid={int(row['content_id'])}  dom={row['domain']}  sim={row['score']:.4f}")
    except Exception:
        pass

if __name__ == "__main__":
    main()


✅ 아이템 임베딩 로드: item_embeddings_torch.csv (items=139, dim=64)
✅ recommendations_per_domain.csv 저장: C:\Users\LG\Desktop\2025-2\A.O.D\db_export\clean\recommendations_per_domain.csv (rows=90)

[User 1] 총 9개
  # 1  cid=57  dom=AV  sim=0.3405
  # 2  cid=86  dom=AV  sim=0.3375
  # 3  cid=49  dom=AV  sim=0.3046
  # 4  cid=207  dom=WEBNOVEL  sim=0.2554
  # 5  cid=232  dom=WEBNOVEL  sim=0.2549
  # 6  cid=122  dom=GAME  sim=0.2396
  # 7  cid=131  dom=WEBNOVEL  sim=0.2148
  # 8  cid=123  dom=GAME  sim=0.1827
  # 9  cid=121  dom=GAME  sim=0.1629

[User 2] 총 9개
  # 1  cid=232  dom=WEBNOVEL  sim=0.4273
  # 2  cid=49  dom=AV  sim=0.4104
  # 3  cid=222  dom=WEBNOVEL  sim=0.3949
  # 4  cid=65  dom=AV  sim=0.3864
  # 5  cid=8  dom=AV  sim=0.3794
  # 6  cid=240  dom=WEBNOVEL  sim=0.3548
  # 7  cid=113  dom=GAME  sim=0.1971
  # 8  cid=123  dom=GAME  sim=0.1736
  # 9  cid=122  dom=GAME  sim=0.1426

[User 3] 총 9개
  # 1  cid=123  dom=GAME  sim=0.4318
  # 2  cid=49  dom=AV  sim=0.4039
  # 3  cid=207  dom=WEBNOV